<a href="https://colab.research.google.com/github/kaihcodes/netflix_data_analysis_in_python/blob/main/netflix_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

#there are 5 tabs in this file
df_title = pd.read_excel('https://public.tableau.com/app/sample-data/netflix_titles.xlsx', sheet_name = 'netflix_titles') #there is a show ID field which can be used to merge with other tables
df_director = pd.read_excel('https://public.tableau.com/app/sample-data/netflix_titles.xlsx', sheet_name = 'netflix_titles_directors')
df_country = pd.read_excel('https://public.tableau.com/app/sample-data/netflix_titles.xlsx', sheet_name = 'netflix_titles_countries')
df_cast = pd.read_excel('https://public.tableau.com/app/sample-data/netflix_titles.xlsx', sheet_name = 'netflix_titles_cast') #this might have multiple records for 1 id
df_category = pd.read_excel('https://public.tableau.com/app/sample-data/netflix_titles.xlsx', sheet_name = 'netflix_titles_category')

df_title.sample(15)
df_title.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration_minutes  4267 non-null   object 
 1   duration_seasons  1971 non-null   object 
 2   type              6235 non-null   object 
 3   title             6235 non-null   object 
 4   date_added        6223 non-null   object 
 5   release_year      6234 non-null   float64
 6   rating            6223 non-null   object 
 7   description       6233 non-null   object 
 8   show_id           6232 non-null   float64
dtypes: float64(2), object(7)
memory usage: 438.6+ KB


In [2]:
#title file has a lot of NAs, especially duration seasons field, imputation not feasible, best to remove, 3 step process
#drop duration seasons first, then remove na records but excluding duration minutes column, then impute duration minutes na records with avg
df_title1 = df_title.copy()
df_title1.drop(columns='duration_seasons',axis=1,inplace=True)
df_title1.info()

df_title1
df_title1.dropna(subset = ['type','title','date_added','release_year','rating','description','show_id'],inplace=True)
df_title1.info()

df_title1['duration_minutes'].mean(skipna=True)
df_title1.head(20)
df_title1['duration_minutes'].fillna(value=df_title1['duration_minutes'].mean(skipna=True),inplace=True)
df_title1.head(20)
df_title1.info()
#now there is no na in the dataset


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration_minutes  4267 non-null   object 
 1   type              6235 non-null   object 
 2   title             6235 non-null   object 
 3   date_added        6223 non-null   object 
 4   release_year      6234 non-null   float64
 5   rating            6223 non-null   object 
 6   description       6233 non-null   object 
 7   show_id           6232 non-null   float64
dtypes: float64(2), object(6)
memory usage: 389.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6212 entries, 0 to 6224
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration_minutes  4255 non-null   object 
 1   type              6212 non-null   object 
 2   title             6212 non-null   object 
 3   date_added        6212 non-nu

<ipython-input-2-96df9a0f2a3a>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_title1['duration_minutes'].fillna(value=df_title1['duration_minutes'].mean(skipna=True),inplace=True)
<ipython-input-2-96df9a0f2a3a>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_title1['duration_minutes'].fillna(value=df

In [19]:
#want to merge these tables
df_director.head(20)
df_director.info()
df_title1['show_id'].value_counts() #unique show id
df_director['show_id'].value_counts() #there are duplicates in this table

#check duplicates, about 1000 records, mutliple directors for a movie, makes sense
df_director[df_director.duplicated(subset='show_id',keep=False)].sort_values('show_id')

df_director1 = df_director.copy()
df_director1 = df_director1.groupby('show_id').agg({'director': lambda x: x.tolist()}).rename(columns={'director': 'director'}).reset_index() #use reset index to turn director into a column
df_director1.info()
df_director1.head(20)

#now we are ready to join
#shoudl be 1 to 1, check
df_title1['show_id'].nunique() #matches total record, no duplicate
df_title1.info() #6212
df_director1['show_id'].nunique() #same here, ready to merge
df_director1.info() #4265
df_merged = pd.merge(df_title1,df_director1,how='left', on='show_id')
df_merged.head(20)
df_merged.info() #4257 values instead of 4265, meaning not all records from director dataframe is merged
#check which records didn't get merged, should return aprox 8 records
df_merged1 = df_merged.loc[df_merged['director'].notna(), ['show_id']]
df_merged2 = pd.merge(df_director1,df_merged1,how='left', on='show_id',indicator=True)
df_merged2.info()
df_merged2.loc[df_merged2['_merge'] == 'left_only']



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4852 entries, 0 to 4851
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   director  4852 non-null   object
 1   show_id   4852 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 75.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4265 entries, 0 to 4264
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   show_id   4265 non-null   int64 
 1   director  4265 non-null   object
dtypes: int64(1), object(1)
memory usage: 66.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6212 entries, 0 to 6224
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration_minutes  6212 non-null   float64
 1   type              6212 non-null   object 
 2   title             6212 non-null   object 
 3   date_added        6212 non-null   object 

,show_id,director,_merge
702,70129452,[Louis C.K.],left_only
1481,80092835,[George Ford],left_only
1483,80092839,[George Ford],left_only
1721,80114111,[Louis C.K.],left_only
1804,80119194,[William Wyler],left_only
2000,80144119,[Alessandro Pepe],left_only
2206,80161109,[Louis C.K.],left_only
2524,80188902,[Elliot Hegarty],left_only


In [30]:
#continue merge country table
df_country.info() #7179 records
df_country['show_id'].nunique() #5758 unique records
df_country[df_country.duplicated(subset='show_id',keep=False)].sort_values('show_id') # a lot of international co production i guess
df_merged.loc[df_merged['show_id'] == 347365]
#looking at 1 example here, Candyman in 1992, country of origin is US and UK, verified it on IMDB, interesting
df_country1 = df_country.copy()
df_country1.info()
df_country1 = df_country1.groupby('show_id').agg({'country': lambda x: x.tolist()}).reset_index()
df_country1.info()
df_country1.head(50)

df_merged_new = pd.merge(df_merged,df_country1,how='left', on='show_id')
df_merged_new.sample(20)
df_merged_new.info() #5741 country records got matched, compared 5758, about 17 records didn't get matched, kind of like director


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7179 entries, 0 to 7178
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  7179 non-null   object
 1   show_id  7179 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7179 entries, 0 to 7178
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  7179 non-null   object
 1   show_id  7179 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5758 entries, 0 to 5757
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   show_id  5758 non-null   int64 
 1   country  5758 non-null   object
dtypes: int64(1), object(1)
memory usage: 90.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6212 entries, 0 to 6211
Data colu